In [10]:
from langchain_ollama import OllamaLLM, OllamaEmbeddings
from langchain_core.prompts import ChatPromptTemplate 
from langchain_community.vectorstores import Chroma
import pandas as pd

load document

In [11]:
df = pd.read_csv("realistic_restaurant_reviews.csv")
df


,Title,Date,Rating,Review
0,Best pizza in town,2024-03-15,5,The crust was perfectly crispy on the outside ...
1,Disappointed with service,2024-02-20,2,"While the pizza itself was decent, we waited o..."
2,Authentic Italian experience,2024-01-05,5,This place reminds me of the pizzerias in Napl...
3,Overpriced for what you get,2024-03-01,3,$24 for a medium pizza with just two toppings ...
4,Great gluten-free option,2024-02-15,4,"As someone with celiac disease, finding good g..."
...,...,...,...,...
118,Perfect for dietary restrictions,2024-02-09,5,"As someone with multiple food allergies, I app..."
119,Tasteless cheese substitute,2024-03-01,2,Their vegan cheese option was terrible - had n...
120,Masterclass in proper baking,2024-01-10,5,Watching the pizzaiolo work the oven is like w...
121,Tomato sauce tasted rancid,2024-02-16,1,The sauce had a distinctly off flavor that sug...


In [12]:
documents = [
    f"Title: {row['Title']}\nDate: {row['Date']}\nRating: {row['Rating']}\nReview: {row['Review']}"
    for _, row in df.iterrows()
]

Create embeddings and vectorstore

In [13]:
embeddings = OllamaEmbeddings(model="mxbai-embed-large")
vectorstore = Chroma.from_texts(documents, embedding=embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

Initialize LLM

In [14]:
llm = OllamaLLM(model="llama3.2", temperature=0.3)


Build the RetrievalQA Chain

In [15]:
from langchain.chains import RetrievalQA

# retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    retriever = retriever,
    chain_type = "stuff"

)

In [17]:
query = "What is the best pizza place mentioned in the reviews and best rating?"
answer = qa_chain.run(query)
print(answer)

The best pizza place mentioned in the reviews with a 5-star rating is not explicitly stated as a single location, but rather it appears to be the same place multiple times. The review titles are all "Best [insert type of pizza] in town", suggesting that this is a highly-recommended spot for different types of pizzas.

However, based on the repeated mention of their signature pepperoni pizza and the 'White Album' pizza, it can be inferred that these two pizzas are likely among their most popular options.


In [18]:
qa_chain.run("Summarize all negative reviews.")
qa_chain.run("Which review mentioned wood-fired Margherita pizza?")
qa_chain.run("What is the overall rating trend in February 2024?")


"Based on the provided reviews, it appears that there are multiple reviewers with similar ratings of 3 out of 5 stars. However, I don't have enough information to determine if there's a clear trend or pattern in the ratings for February 2024. The reviews seem to be from different dates and don't provide any further context about the overall rating trend for that specific month."